In [18]:
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 0.13.277.1-preview

In [19]:
#!import Skills/CustomSkill/CompanySearchSkill.cs

In [20]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;

In [21]:

using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning.Planners;

In [22]:
IKernel kernel = Kernel.Builder.Build();

In [23]:
kernel.Config.AddAzureTextCompletionService(
    "davinci",      
    "GPT3Model",                  
    "Your Endpoint",    
    "Your Key"       
);

In [24]:
var planner = new SequentialPlanner(kernel);

In [25]:
// var planner = kernel.ImportSkill(new SequentialPlanner(kernel));

In [26]:
planner

Microsoft.SemanticKernel.Planning.Planners.SequentialPlanner

In [27]:

CompanySearchSkill customSkill = new CompanySearchSkill();

In [28]:
var custom_skill = kernel.ImportSkill (customSkill, "CustomSkill");

In [29]:
var skillsDirectory =  System.IO.Directory.GetCurrentDirectory() + "/Skills";

In [30]:
var write_skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory,"WriteSkill");
var email_skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory,"EmailSkill");

In [31]:
var plan = await planner.CreatePlanAsync("查找广州天气把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件");
// var ask = "查找广州天气，把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件";
// var plan = await planner.CreatePlanAsync(ask);
// var plan = await kernel.RunAsync(ask, planner["CreatePlan"]);

In [36]:
// plan.Variables.ToPlan().PlanString
plan.Steps

index value 0 Microsoft.SemanticKernel.Orchestration.Plan State index value 0 [INPUT, ] Key INPUT Value Steps NamedParameters index value 0 [INPUT, Guangzhou] Key INPUT Value Guangzhou NamedOutputs index value 0 [CITY_NAME, ] Key CITY_NAME Value 1 [INPUT, ] Key INPUT Value HasNextStep False NextStepIndex 0 Name EmployeeSearch SkillName CustomSkill Description search employee infomation IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences 1 Microsoft.SemanticKernel.Orchestration.Plan State index value 0 [INPUT, ] Key INPUT Value Steps NamedParameters index value 0 [INPUT, $CITY_NAME] Key INPUT Value $CITY_NAME NamedOutputs index value 0 [WEATHER_RESULT, ] Key WEATHER_RESULT Value 1 [INPUT, ] Key INPUT Value HasNextStep False NextStepIndex 0 Name WeatherSearch SkillName CustomSkill Description search weather IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences 2 Microsoft.SemanticKernel.Orchestration.Plan State index value 0 [INPUT, ] Key INPUT Value Steps NamedParameters index value 0 [INPUT, ] Key INPUT Value NamedOutputs index value 0 [INPUT, ] Key INPUT Value HasNextStep False NextStepIndex 0 Name SkillName Microsoft.SemanticKernel.Orchestration.Plan Description IsSemantic False RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 256 StopSequences 3 Microsoft.SemanticKernel.Orchestration.Plan State index value 0 [INPUT, ] Key INPUT Value Steps NamedParameters index value 0 [label, Weather] Key label Value Weather 1 [INPUT, $TRANSLATED_WEATHER_RESULT] Key INPUT Value $TRANSLATED_WEATHER_RESULT NamedOutputs index value 0 [WEATHER_LABEL, ] Key WEATHER_LABEL Value 1 [INPUT, ] Key INPUT Value HasNextStep False NextStepIndex 0 Name Classification SkillName WriteSkill Description 文本分类 IsSemantic True RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 60 StopSequences 4 Microsoft.SemanticKernel.Orchestration.Plan State index value 0 [INPUT, ] Key INPUT Value Steps NamedParameters index value 0 [INPUT, $WEATHER_LABEL] Key INPUT Value $WEATHER_LABEL NamedOutputs index value 0 [TIPS_RESULT, ] Key TIPS_RESULT Value 1 [INPUT, ] Key INPUT Value HasNextStep False NextStepIndex 0 Name Tips SkillName WriteSkill Description 穿衣提示 IsSemantic True RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0.5 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 512 StopSequences 5 Microsoft.SemanticKernel.Orchestration.Plan State index value 0 [INPUT, ] Key INPUT Value Steps NamedParameters index value 0 [INPUT, $TRANSLATED_WEATHER_RESULT;$TIPS_RESULT] Key INPUT Value $TRANSLATED_WEATHER_RESULT;$TIPS_RESULT NamedOutputs index value 0 [RESULT__EMAIL_RESULT, ] Key RESULT__EMAIL_RESULT Value 1 [INPUT, ] Key INPUT Value HasNextStep False NextStepIndex 0 Name WeatherMail SkillName EmailSkill Description Write an email about weather IsSemantic True RequestSettings Microsoft.SemanticKernel.AI.TextCompletion.CompleteRequestSettings Temperature 0 TopP 0 PresencePenalty 0 FrequencyPenalty 0 MaxTokens 1024 StopSequences

In [38]:
plan.ToJson()

{"state":[{"Key":"INPUT","Value":""}],"steps":[{"state":[{"Key":"INPUT","Value":""}],"steps":[],"named_parameters":[{"Key":"INPUT","Value":"Guangzhou"}],"named_outputs":[{"Key":"CITY_NAME","Value":""},{"Key":"INPUT","Value":""}],"next_step_index":0,"name":"EmployeeSearch","skill_name":"CustomSkill","description":"search employee infomation"},{"state":[{"Key":"INPUT","Value":""}],"steps":[],"named_parameters":[{"Key":"INPUT","Value":"$CITY_NAME"}],"named_outputs":[{"Key":"WEATHER_RESULT","Value":""},{"Key":"INPUT","Value":""}],"next_step_index":0,"name":"WeatherSearch","skill_name":"CustomSkill","description":"search weather"},{"state":[{"Key":"INPUT","Value":""}],"steps":[],"named_parameters":[{"Key":"INPUT","Value":""}],"named_outputs":[{"Key":"INPUT","Value":""}],"next_step_index":0,"name":"","skill_name":"Microsoft.SemanticKernel.Orchestration.Plan","description":""},{"state":[{"Key":"INPUT","Value":""}],"steps":[],"named_parameters":[{"Key":"label","Value":"Weather"},{"Key":"INPUT"

In [39]:
var result = await kernel.RunAsync(plan);

In [40]:
result.Result

尊敬的同事：

您好！

根据最新的天气预报，未来几天的天气将会变得更加寒冷，因此，我建议您穿上深色的休闲衣服，搭配一双舒适的运动鞋，以便您在出行时能够更加舒适。

此外，请您随时留意出行状况，以免受到不必要的影响。如果您有任何疑问，请随时与我联系。

祝您出行愉快！

此致

敬礼